In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
import gzip

In [3]:
pd.read_csv?

In [12]:
people = pd.read_csv("data/people.csv", parse_dates=['date'])
train = pd.read_csv("data/act_train.csv", parse_dates=['date'])
test = pd.read_csv("data/act_test.csv", parse_dates=['date'])

In [13]:
people['year'] = people['date'].dt.year
people['month'] = people['date'].dt.month
people['day'] = people['date'].dt.day
people.drop('date', axis=1, inplace=True)

train['set'] = 'train'
test['set'] = 'test'
act_data = pd.concat([train, test], axis=0)
act_data['year'] = act_data['date'].dt.year
act_data['month'] = act_data['date'].dt.month
act_data['day'] = act_data['date'].dt.day
act_data.drop('date', axis=1, inplace=True)
del train, test

In [14]:
people_cat_vars = ['char_1', 'group_1', 'char_2', 'char_3', 'char_4',
                   'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 
                   'char_10', 'char_11', 'char_12', 'char_13', 'char_14', 
                   'char_15', 'char_16', 'char_17', 'char_18', 'char_19', 
                   'char_20', 'char_21', 'char_22', 'char_23', 'char_24', 
                   'char_25', 'char_26', 'char_27', 'char_28', 'char_29',
                   'char_30', 'char_31', 'char_32', 'char_33', 'char_34', 
                   'char_35', 'char_36', 'char_37']

act_cat_vars = ['activity_category', 'char_1', 'char_10', 
                'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 
                'char_8', 'char_9']

In [15]:
le = LabelEncoder()
for var in people_cat_vars:
    people[var] = le.fit_transform(people[var].values)
for var in act_cat_vars:
    act_data[var].fillna('na', inplace=True)
    act_data[var] = le.fit_transform(act_data[var].values)

In [16]:
data = pd.merge(act_data, people, how='left', on='people_id')
del people

In [17]:
train = data.loc[data['set'] == 'train'].drop('set', axis=1)
train['outcome'] = train['outcome'].astype(np.int8)
test = data.loc[data['set'] == 'test'].drop(['set', 'outcome'], axis=1)

In [18]:
features = list(train.columns)
for var in ['activity_id', 'outcome', 'people_id']:
    features.remove(var)
    
target = 'outcome'

In [19]:
pickle_file = 'processedData.pgz'

with gzip.open(pickle_file, 'wb') as f:
    save = {
        'train': train,
        'test': test,
        'features': features,
        'target': target
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
del train, test